In [1]:
import requests
import json
import pickle
import re
import pandas as pd
import numpy as np
from copy import deepcopy
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook as tqdm

In [3]:
wacraft_server_activity = pickle.load(open("crawled_server_activity.p", 'rb'))

In [ ]:
from clean_server_activity import clean
clean()

In [35]:
cleaned_series = pickle.load(open('cleaned_server_activity.p', 'rb'))

# Sort servers by timezone

In [ ]:
blizz_realms_us = pickle.load(open('blizz_realms_us.p', 'rb'))['realms']

In [ ]:
blizz_realms_and_timezone_us = [(blizz_realms_us[i]['name'], blizz_realms_us[i]['timezone']) 
                                for i in range(len(list(blizz_realms_us)))]
blizz_realms_and_timezone_us = pd.DataFrame(blizz_realms_and_timezone_us, columns=['server_name', 'timezone'])

In [ ]:
timezone_dict = blizz_realms_and_timezone_us.groupby("timezone").groups

In [ ]:
# Create a dict with timezone as key and server names as values
timezone_dict_to_convert = deepcopy(timezone_dict)
timezone_dict = {}
for key in timezone_dict_to_convert.keys():
    timezone_dict[key] = blizz_realms_and_timezone_us.iloc[timezone_dict_to_convert[key].tolist()]['server_name'].tolist()

In [ ]:
matched_server = set(list(server_names_warcraft_realms_and_dfs.keys())) & set(blizz_realms_and_timezone_us['server_name'].tolist())

In [ ]:
averaged_timezone_dict = deepcopy(timezone_dict)
for timezone in averaged_timezone_dict:
    running_avg = pd.DataFrame([(np.zeros(24)),(np.zeros(24))], index=['alliance_avg', 'horde_avg']).T
    try:
        for server in averaged_timezone_dict[timezone]:
            running_avg += server_names_warcraft_realms_and_dfs[server]
        averaged_timezone_dict[timezone] = running_avg/len(averaged_timezone_dict[timezone])
    except KeyError:
        pass

In [ ]:
averaged_timezone_dict

In [ ]:
a = pd.DataFrame([(1,2),(np.nan,2)],columns =['lol','rofl'])
a

In [ ]:
a.sum(axis=1)

In [ ]:
realms_us_dict = {"name":[],
                 "timezone": [],
                 'dataframe': []}
for servername, timezone in blizz_realms_and_timezone_us:
    if server_timezone_tuple[0] in matched_server:
        realms_us_dict["name"].append(servername)
        realms_us_dict["timezone"].append(timezone)
        
        realms_us_dict["dataframe"].append(server_names_warcraft_realms_and_dfs[server_timezone_tuple[0]])     

In [ ]:
df = pd.DataFrame.from_dict(realms_us_dict)

In [ ]:
realms_eu = requests.get("https://eu.api.blizzard.com/wow/realm/status?access_token=USrQ2Vy3vjvTs2wAq1rZrF0WQFd26p8J0j")
realms_eu = realms_eu.content.decode('utf-8')
realms_eu = json.loads(realms_eu)
realms_eu = realms_eu['realms']

In [ ]:
eu_keys = set([realms[i]['locale'] for i in range(len(realms))])
eu_server = {}
for key in eu_keys:
    eu_server[key] = []
eu_server
for i in range(len(realms_eu)):
    eu_server[realms_eu[i]['locale']].append(realms_eu[i]['name'])

In [ ]:
# Match with ID
id_per_name = requests.get("https://eu.api.blizzard.com/data/wow/realm/?namespace=dynamic-eu&locale=en_US&access_token=USrQ2Vy3vjvTs2wAq1rZrF0WQFd26p8J0j")
id_per_name = json.loads(id_per_name.content.decode('utf-8'))['realms']


In [ ]:
name_id_dict = {}
for server in id_per_name:
    name_id_dict[server['name']] = server['id']

In [ ]:
for key in eu_keys:
    print(key, len(eu_server[key]))

In [ ]:
realms_us = requests.get("https://us.api.blizzard.com/wow/realm/status?access_token=USrQ2Vy3vjvTs2wAq1rZrF0WQFd26p8J0j")
realms_us = realms_us.content.decode('utf-8')
realms_us = json.loads(realms_us)
realms_us = realms_us['realms']